In [1]:
import os
import time
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
ai_studio_token = user_secrets.get_secret("GOOGLE_AI_STUDIO")
genai.configure(api_key=ai_studio_token)

In [4]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

In [8]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

In [23]:
from IPython.display import Video
mf_ep = '/kaggle/input/modernfamilyep/Modern.Family.S09E10.No.Small.Feet.720p.WEB-HD.x264-Pahe.in.mkv'

In [17]:
files = [
  upload_to_gemini(mf_ep, mime_type="video/webm"),
]

wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

Uploaded file 'Modern.Family.S09E10.No.Small.Feet.720p.WEB-HD.x264-Pahe.in.mkv' as: https://generativelanguage.googleapis.com/v1beta/files/x6ahio7i7xoz
Waiting for file processing...
......all files ready



In [18]:
response = chat_session.send_message("Please give me the summary of the episode")
print(response.text)

This episode of Modern Family is about Jay's new business, Pritchett's Closets and Blinds. He won an award for it at the Expo Internationale Du Closet in Berlin. Meanwhile, Claire, Gloria, and Phil are trying to sell their house but Gloria keeps sensing bad vibes from the house, which Phil believes is just her being overly dramatic. Cam is dealing with his best friend Pam moving in with him because her boyfriend Bow is in jail. The episode ends with Cam and Mitchell discovering that Bow has come back and is trying to reconcile with Pam. 



In [20]:
response = chat_session.send_message("What did Haley do in this episode")
print(response.text)

Haley was not in this episode of Modern Family. 



In [22]:
response = chat_session.send_message("Please give me the timestaps of Alex and luke of the episode")
print(response.text)

Here are the timestamps for Alex and Luke in this episode:

* **0:00-0:37:** Alex and Luke are at home, and Alex is upset that Luke has been selling her old, smelly sneakers online.
* **0:37-0:49:** Alex and Luke are sitting on the couch, and they're talking about how much money they've made from selling Alex's old shoes.
* **7:42-8:17:** Alex is shaving Luke's legs in the bathroom, and they're talking about how much money they've been making.
* **15:48-16:31:** Alex and Luke arrive at the office and Claire is angry with them for using her FedEx account to ship the shoes. 

Let me know if you have any other questions. 

